In [ ]:
!pip install anchor-exp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from anchor import utils
from anchor import anchor_tabular
import pandas as pd

In [ ]:
# Load your data into a numpy array
train_data = pd.read_csv("FICO.full.csv")
train_data=pd.DataFrame(train_data)
train_data.to_csv("my_data.csv")
#data = pd.read_csv("my_data.csv")
data = np.genfromtxt("my_data.csv", delimiter=",")
data = data[1:]
data = data[:,1:]





In [ ]:
# Split your data into training and testing sets
X_train, y_train = data[:, :-1], data[:, -1]

# Train a random forest model on your training data
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf.fit(X_train, y_train)
X_train.shape

(10459, 17)

In [ ]:
# Define a function that takes an instance and returns its predicted class label
predict_fn = lambda x: rf.predict(x.reshape(1, -1)).ravel()

# Define a function that checks if a given instance has a different class label than the original instance
def has_different_label(x, original_label):
    return rf.predict(x.reshape(1, -1)).ravel() != original_label

# Define a function that searches for a counterfactual instance using the Anchors algorithm
def search_counterfactual(x, original_label):
    # Define the search parameters
    search_params = {
        "sample_whole_training": False,
        "seed": 1
    }

    # Define the feature names and the class names
    feature_names = ["ExternalRiskEstimate<0.49","ExternalRiskEstimate<0.65","ExternalRiskEstimate<0.80","NumSatisfactoryTrades<0.5","TradeOpenTime<0.6","TradeOpenTime<0.85","TradeFrequency<0.45","TradeFrequency<0.6","Delinquency<0.55","Delinquency<0.75","Installment<0.5","Installment<0.7","Inquiry<0.75","RevolvingBalance<0.4","RevolvingBalance<0.6","Utilization<0.6","TradeWBalance<0.33"]
    class_names = ["0", "1"]

    # Create an Anchors explainer object for your random forest model
    explainer = anchor_tabular.AnchorTabularExplainer( class_names,feature_names,
        X_train
        
    )

    # Generate a counterfactual explanation for the given instance using Anchors
    explanation = explainer.explain_instance(
        x, rf.predict, threshold=0.95
    )

    # Search for a counterfactual instance that has a different class label
    
    cfe = explanation.examples(only_different_prediction=True)[0]
    if has_different_label(cfe, original_label):
        return cfe

    return None



In [ ]:
# Define an example instance and its original class label
instance = X_train[0]
original_label = rf.predict(instance.reshape(1, -1)).ravel()[0]
instance



array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.])

In [ ]:
# Search for a counterfactual instance that has a different class label
counterfactual_instance = search_counterfactual(instance, original_label)

if counterfactual_instance is not None:
    print("Counterfactual instance found:", counterfactual_instance)
else:
    print("Counterfactual instance not found.")

Counterfactual instance found: [0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
# https://github.com/marcotcr/anchor/blob/b1f5e6ca37428613723597e85c38558e8cd21c2e/anchor/anchor_explanation.py#L5
# https://arxiv.org/pdf/1805.10820.pdf
# https://github.com/riccotti/LORE/blob/master/test_lore.py

In [ ]:
from prepare_dataset import collectTreesRF, exportTreeCollection

In [ ]:
n_nodesRF, children_leftRF, children_rightRF, featureRF, thresholdRF, node_depthRF, is_leavesRF, nodeValuesRF = collectTreesRF(rf)
exportTreeCollection("Ficofull", "RF",  0, len(X_train[0]), len(rf.classes_),
                             n_nodesRF, children_leftRF, children_rightRF, featureRF, thresholdRF,
                             node_depthRF, is_leavesRF, nodeValuesRF)